In [7]:
import os
print(os.getcwd())

/content


# Encoding

Ya que en el dataset que se nos dio tenian vairiables que en su mayoria son ordinales, el encoding se hizo de manera que se les dio un valor entero a cada uno. Variables que no tienen una ordinalidad estricta como por ejemplo raza se han evaluado en base a que tan comunes son, por ejemplo en este dataset Otro es el mas raro,y por lo tanto el que tiene mayor valor.

In [9]:
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv('drive/MyDrive/Archivos_collab/adultIncome.csv')
df.replace(inplace=True, to_replace=[' Never-worked'],value= 0)
df.replace(inplace=True, to_replace=[' Private'],value= 0)
df.replace(inplace=True, to_replace=[' Local-gov'], value=1)
df.replace(inplace=True, to_replace=[' Federal-gov'], value=2)
df.replace(inplace=True, to_replace=[' Preschool'], value=-1)
df.replace(inplace=True, to_replace=[' HS-grad'], value=0)
df.replace(inplace=True, to_replace=[' Some-college'], value=1)
df.replace(inplace=True, to_replace=[' Bachelors'], value=2)
df.replace(inplace=True, to_replace=[' Prof-school'], value=3)
df.replace(inplace=True, to_replace=[' Masters'], value=4)
df.replace(inplace=True, to_replace=[' Doctorate'],value= 5)
df.replace(inplace=True, to_replace=[' Never-married'], value=0)
df.replace(inplace=True, to_replace=['Married'], value=1)
df.replace(inplace=True, to_replace=[' Divorced'], value=2 )
df.replace(inplace=True, to_replace=[' Widowed'], value=3) 
df.replace(inplace=True, to_replace=[' Black'], value=0)
df.replace(inplace=True, to_replace=[' Asian-Pac-Islander'], value=2)
df.replace(inplace=True, to_replace=[' Amer-Indian-Eskimo'], value=3)
df.replace(inplace=True, to_replace=[' Other'], value=4)
df.replace(inplace=True, to_replace=[' White'], value=1)
df.replace(inplace=True, to_replace=[' Male'], value=0)
df.replace(inplace=True, to_replace=[' Female'],value= 1)
df.replace(inplace=True, to_replace=[' <=50K'], value=0)
df.replace(inplace=True, to_replace=[' >50K'], value=1)
df.to_csv('incomeProcessed.csv', index=False)
df = pd.read_csv('incomeProcessed.csv')
print(df.head())

FileNotFoundError: ignored

# Discretizacion y normalizacion

Ya que tenemos variables como edad y horas por semana trabajadas, con desviaciones estandar mas altas que el resto de nuestras otras variables, queremos normalizarlas para que existan en el mismo rango de valores.

In [ ]:
df=(df-df.min())/(df.max()-df.min())
df.head()

,age,wc,education,marital status,race,gender,hours per week,IncomeClass
0,0.287671,0.0,0.166667,0.666667,0.25,0.0,0.397959,0.0
1,0.150685,0.0,0.500000,0.333333,0.00,1.0,0.397959,0.0
2,0.273973,0.0,0.833333,0.333333,0.25,1.0,0.397959,0.0
3,0.191781,0.0,0.833333,0.000000,0.25,1.0,0.500000,1.0
4,0.342466,0.0,0.500000,0.333333,0.25,0.0,0.397959,1.0


# K-means

In [ ]:
import math
import collections
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import scatter
import pandas as pd
# observaciones tiene la forma de ((x1,y1,z1),(x2,y2,z2))

# findCentroid es una funcion que encuentra el hipotetico centroide de un set de observaciones, encontrando su promedio


def findCentroid(observaciones):
    n_dimensiones = len(observaciones[0])
    centroide = []
    for dimension in range(n_dimensiones):
        centroide.append(0)
        for obs in observaciones:
            current = obs[dimension]
            centroide[dimension] = centroide[dimension] + current

        centroide[dimension] = centroide[dimension]/len(observaciones)
    return centroide


def euclidDistance(coord_list_a, coord_list_b):
    accum = 0
    dimensions = len(coord_list_a)
    for dimension in range(dimensions):
        line = (coord_list_a[dimension] - coord_list_b[dimension]) * \
            (coord_list_a[dimension] - coord_list_b[dimension])
        accum += line
    root = math.sqrt(accum)
    return root

# findNearestMean itera sobre las medias y retorna la que este mas cercana por distancia eculideana


def findNearestMean(observacion, means):
    min_distance = 999999
    n_mean = []
    for mean in means:
        distancia = euclidDistance(observacion, mean)
        if distancia < min_distance:
            n_mean = mean
            min_distance = distancia
    return n_mean

# se retorna un diccionario con las medias como llaves, y la lista de observaciones que la tienen como mas cercana como value


def findAllNearestMeans(observaciones, means):
    observaciones_per_mean = collections.defaultdict(list)
    for observacion in observaciones:
        key = findNearestMean(observacion, means)
        buffer = observaciones_per_mean[tuple(key)]
        buffer.append(observacion)
    return observaciones_per_mean

# RandomK es un procedimiento simple que elije un sample aleatorio de los que se tienen; y los inicializa como means.


def generateRandomK(k, observaciones):
    medias = random.sample(observaciones, k)
    return medias


def plotMeans2d(dict_medias):
    dimensiones = [[], [], []]
    media_colors = {}
    unique_medias = list(set(dict_medias.keys()))
    step_size = (256**3)//len(unique_medias)
    for i, m in enumerate(unique_medias):
        media_colors[m] = '#{}'.format(hex(step_size*i)[2:])
    for media in dict_medias:
        for observacion in dict_medias[media]:
            dimensiones[2].append(tuple(media))
            for dimension in range(len(observacion)):
                dimensiones[dimension].append(observacion[dimension])
    print('el pepe')
    print(dimensiones[2])
    colors = [media_colors[current_media] for current_media in dimensiones[2]]
    for item in range(len(colors)):
        if colors[item] == '#0':
            colors[item] = '#000000'
    #tr = plt.scatter(dimensiones[0], dimensiones[1],  alpha=0.5)
    tr = plt.scatter(dimensiones[0], dimensiones[1], c=colors, alpha=0.5)
    plt.show()


def kmeans(k, observaciones):
    # encontramos la configuracion inicial
    medias = generateRandomK(k, observaciones)
    cercanas = findAllNearestMeans(observaciones, medias)
    # se sigue iterando hasta que se deje de cambiar la media despues de una iteracion
    while True:
        # cada loop se usa la lista de centroides para ver las medias updateadas
        centroides = []
        
        for media in medias:
            # para cada cluster, se encuentra su centroide
            
            c = cercanas[tuple(media)]
            centroides.append(findCentroid(c))
        if medias == centroides:
            break
        medias = centroides
        cercanas = findAllNearestMeans(observaciones, medias)
    return medias


def dfToList(df):
    lista = []
    for row in df.itertuples():
        lista.append(list(row)[1:])
    return lista

mapa = dfToList(df)
result = kmeans(4, mapa)
print (result)

[[0.25112790008948255, 0.06606485740570377, 0.29410073597055963, 0.2356255749769893, 0.24899379024839005, 0.0, 0.4088778795786538, 0.0], [0.362963930673238, 0.11137602179836512, 0.44883439297608024, 0.3339388434756177, 0.25039736603088103, 0.0, 0.46000157556210997, 1.0], [0.1648638203832361, 0.061490002738975626, 0.3309139048662465, 0.04720167990504899, 0.23520953163516845, 1.0, 0.3566174210029201, 0.001369487811558477], [0.3832548233977678, 0.12779973649538867, 0.3482652613087393, 0.6181379007466066, 0.23641304347826086, 1.0, 0.38882390901025365, 0.2644927536231884]]


# KNN

# Supervisado y comparaciones
